# Explore Curation Events

In [295]:
# standard imports
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
from eth_utils import keccak
#from web3 import Web3

In [2]:
# function used to query each type of event from The Graph
def query_theGraph(raw_query, field_name, url, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}"
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:blockNumber, where:{blockNumber_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:blockNumber, where: {blockNumber_gt: ' + indexer + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['blockNumber']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [4]:
#get allocationCreated event
url = 'https://api.thegraph.com/subgraphs/name/fordn/graph-network-events'

created_query = '''query{
  nsignalMinteds() {
    id
    blockNumber
    blockTime
    eventTime
    graphAccount
    subgraphNumber
    nameCurator
    nSignalCreated
    vSignalCreated
    tokensDeposited
  }  
}
'''

field_name = 'nsignalMinteds'

curation_mint_df = query_theGraph(created_query, field_name, url, False)

In [8]:
#save allocationCreated data
curation_mint_df['type'] = 'curationMinted'
#alloc_created_df.to_csv('allocationCreateds.csv', index = False, header = True)
curation_mint_df.head()

,id,blockNumber,blockTime,eventTime,graphAccount,subgraphNumber,nameCurator,nSignalCreated,vSignalCreated,tokensDeposited,type
0,0x4c1060853752b5221b1e27c2264034d0bd5fd86c5267...,11474121,1608254015,1608254015,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,279284800875378823397,279284800875378823397,80000000000000000000000,curationMinted
1,0xea7dbe2f6164530a53ac8cf9d99b46993f44d8ca8258...,11476852,1608289893,1608289893,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x0f3cf18b37c55f5a319d6cae8d8ba114cb094af0,627949250367270370,627949250367270370,360151469700892136118,curationMinted
2,0xfbdb37e510284ebbeb45fbfdf28191219faf84bc31bf...,11476875,1608290226,1608290226,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x0f3cf18b37c55f5a319d6cae8d8ba114cb094af0,125503147008134479971,125503147008134479971,88216309708626528177242,curationMinted
3,0xfbda17a348fb979ad36e27445d656b23458a938fadd2...,11476931,1608291049,1608291049,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0xb6916933aa0bac4b6d09260719c4055363642745,89477475325111327822,89477475325111327822,82622974825605383769993,curationMinted
4,0x395da0c992967e085a68fd69825c8f629486ef10d48b...,11531681,1609017205,1609017205,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x849ed8601451dcac8012425ccd56fd1339db498c,1568268903568881,1568268903568881,1000000000000000000,curationMinted


In [10]:
#get allocationCreated event
url = 'https://api.thegraph.com/subgraphs/name/fordn/graph-network-events'

created_query = '''query{
  nsignalBurneds() {
    id
    blockNumber
    blockTime
    eventTime
    graphAccount
    subgraphNumber
    nameCurator
    nSignalBurnt
    vSignalBurnt
    tokensReceived
  }  
}
'''

field_name = 'nsignalBurneds'

curation_burned_df = query_theGraph(created_query, field_name, url, False)

In [11]:
#save allocationCreated data
curation_burned_df['type'] = 'curationBurned'
#alloc_created_df.to_csv('allocationCreateds.csv', index = False, header = True)
curation_burned_df.head()

,id,blockNumber,blockTime,eventTime,graphAccount,subgraphNumber,nameCurator,nSignalBurnt,vSignalBurnt,tokensReceived,type
0,0xf5fd904385312b521779beb91a757a403132ebf7bbf7...,11486499,1608418200,1608418200,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0xb6916933aa0bac4b6d09260719c4055363642745,89477475325111327822,89477475325111327822,80557400454965249175496,curationBurned
1,0x7b0d39ca37ba86fd5e6ecc4433513ffba4b853a90bd2...,11491383,1608482858,1608482858,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x0f3cf18b37c55f5a319d6cae8d8ba114cb094af0,95,95,77029,curationBurned
2,0xe46b8054c269f1c5811dc5cdd0d4564bb92f370477df...,11491393,1608482989,1608482989,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x0f3cf18b37c55f5a319d6cae8d8ba114cb094af0,95000000000000000000,95000000000000000000,68004020455437308993046,curationBurned
3,0x620c0af5345f21ec87c9b73864d3709d735fb7aac155...,11531871,1609019622,1609019622,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x849ed8601451dcac8012425ccd56fd1339db498c,1568268903568881,1568268903568881,975006929934390676,curationBurned
4,0x10afff8568c6e870a7b596787cb284e43842892e3a74...,12318426,1619473178,1619473178,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,1,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,1000000000000000000,1000000000000000000,545931348525571351761,curationBurned


In [12]:
#get allocationCreated event
url = 'https://api.thegraph.com/subgraphs/name/fordn/graph-network-events'

created_query = '''query{
  grtwithdrawns() {
    id
    blockNumber
    blockTime
    eventTime
    graphAccount
    subgraphNumber
    nameCurator
    nSignalBurnt
    withdrawnGRT
  }  
}
'''

field_name = 'grtwithdrawns'

curation_withdraw_df = query_theGraph(created_query, field_name, url, False)

In [15]:
#save allocationCreated data
curation_withdraw_df['type'] = 'curationWithdraw'
#alloc_created_df.to_csv('allocationCreateds.csv', index = False, header = True)
curation_withdraw_df.head()

,id,blockNumber,blockTime,eventTime,graphAccount,subgraphNumber,nameCurator,nSignalBurnt,withdrawnGRT,type
0,0x6b79238a65b76c0c34c3c1d80baf336d2d8825a56dca...,12323378,1619538992,1619538992,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,3,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,3122498999199199102,9750000000000000000,curationWithdraw
1,0x949a0846530d16ab2d7752a835e35e4ad3dea65795af...,12342271,1619791259,1619791259,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,0,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,147529798656557600,305378117070384881711,curationWithdraw
2,0x43ca80f165479c0c3e845a99c0cf607a762158c96d87...,12788295,1625768410,1625768410,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0xc6ba3e757f3af487e388230f1e3a40615ea24002,221040135439528302402,117649984549175437607963,curationWithdraw
3,0xbd3708e37001acedcab6e329a37401920daff88caedd...,12788326,1625768790,1625768790,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,279284800875378823397,148651069374670374373841,curationWithdraw
4,0xa2f3f0f3ed3084b57e8cfbf3a2e156f000657974c9db...,12788408,1625769908,1625769908,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0xc18d03a8be67cb08c01434861dc561a70c4b3aa8,64571405224668812,34368531361487313663,curationWithdraw


In [300]:
indexer_df = pd.read_csv('../another_indexer/all_events/combinedEvents.csv', low_memory = False)
indexer_df.head()

,allocationID,blockNumber,epoch,id,indexer,logIndex,subgraphDeploymentID,tokens,type,isDelegator,...,from,rebateFees,delegator,shares,eventTime,until,indexingRewardCut,queryFeeCut,delegationFees,amount
0,0xfbc5e8d942122bbad88f4868500356bc59164c8f,11474062,4.0,0xc037be198c842da82ef12c8bc6003e5c8684c1bfa274...,0xa959b5afe73c6faa803541b5c4edc0492dfda294,290,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,8000000000000000000000,allocationCreateds,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0x566bb471e1dec5d97985981b86503dd4be561479,11474065,4.0,0x1fd0fa14121d920bde44890feece6cbb592f064e8ed8...,0xa959b5afe73c6faa803541b5c4edc0492dfda294,297,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,8000000000000000000000,allocationCreateds,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0x4dede41c1cfbaaf990e736039a0ebe1001073f7c,11474097,4.0,0xf470c9f4df65a955622ddb8813334d19f5bb7abf55ee...,0x62a0bd1d110ff4e5b793119e95fc07c9d1fc8c4a,201,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,50000000000000000000000,allocationCreateds,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0xd60a65b8f4b5083a9b8a26775bbf1da7f53c7eae,11474187,4.0,0x5dcc3cc5d2e6fb45fa4a020e77bef9137aa44de13c10...,0x62a0bd1d110ff4e5b793119e95fc07c9d1fc8c4a,50,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,70000000000000000000000,allocationCreateds,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0x7901c76b0ce0db5b3f3e96535b70a7550adcb423,11474266,4.0,0x2a612e88f706256179474aa3b1c09a0c884a2257e911...,0x62a0bd1d110ff4e5b793119e95fc07c9d1fc8c4a,159,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,500000000000000000000000,allocationCreateds,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [301]:
subgraph_df = indexer_df[indexer_df['subgraphDeploymentID'].isna() != 1]

In [302]:
unique_subgraphs = subgraph_df['subgraphDeploymentID'].unique()
unique_subgraphs

array(['0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d60e4243bd3b4e8af7500',
       '0xbbde25a2c85f55b53b7698b9476610c3d1202d88870e66502ab0076b7218f98a',
       '0x666d78959706d7e1ed20befeb1d2c9a4e512a22931d8e22928b1f63ce48fcb40',
       '0xbae01a063ad846d7e4200757480544b5feb1f0217d94db1db97b48bb7f124dbf',
       '0xc420d79a888eb0e34203bfeca1048d35f9763b9880a94922249096e4c9a191c7',
       '0xa7d9d3a173d3d62ce3d05c2d29f3447b04edc248cbcfdb49b205e14247374947',
       '0x2aaeb45afa62af1cc8f6d61decd5249429e0aaed67996238adc2bfd396b5f715',
       '0x014e8a3184d5fad198123419a7b54d5f7c9f8a981116462591fbb1a922c39811',
       '0x5e48ee6773576d45a38934cb19780b5c8c75f8c36e653e234ba5b803746d8f78',
       '0x31f6fb67c823a07a93bae6b022798eebdf86210033c0e97a1402795c0c2fa99b',
       '0x500a8e47cbdeca7386448ae9e7d52578871b9942ebbef4892469da293bb661f9',
       '0x4a008334f8e7ea21be223d15462084f51abcc0e6625f61de806b8456e67e603e',
       '0x4810cc7741423cec7e44fc7ec7a479e521453cbf090e74b6186d3a2264588fe5',

In [303]:
i = 0
 
h = hex(i)
print(type(h))    # '0xfff'

<class 'str'>


In [304]:
0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd
0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d60e4243bd3b4e8af7500
0x0ec8d6a09b0a4de24f3629855d1e6589b0974abf04f5f95399f355c8dd6f8f63

6687230596758730352712405091149319033639655386203023348226781835593757200227

In [305]:
curation_mint_df.iloc[1000]['subgraphDeploymentID']

'0xa765bf6641d71b8887d09b4aa6fce8b237abdef5631415d539b7cda99bc08411'

In [317]:
curation_mint_df.iloc[23]['graphAccount']

'0x26cf67040678eb0f5654c9cbaad78dc1694cbafa'

In [316]:
curation_mint_df.iloc[24]['graphAccount']

'0x0a0319671f2d3c18fb55ab555b48bc01f27747a4'

In [326]:
account = hex(int(curation_mint_df.iloc[24]['graphAccount'][2:], base=16))
graphNum = hex(int(curation_mint_df.iloc[24]['subgraphNumber']))[2:]
number = graphNum.zfill(64)
unhashed_str = hex(int((account + number), base=16))
print(unhashed_str)

0xa0319671f2d3c18fb55ab555b48bc01f27747a40000000000000000000000000000000000000000000000000000000000000000


In [341]:
# convert graphAccount to subgraphDeploymentID
def account_to_id(df):
    account = hex(int(df['graphAccount'], 0))
    graphNum = hex(int(df['subgraphNumber'], 0))[2:]
    number = graphNum.zfill(64)
    unhashed_str = hex(int(account + number, 0))[2:]
    print(unhashed_str)
    if len(unhashed_str) % 2 != 0:
         unhashed_str = '0' + unhashed_str
    unhashed_byte = bytes.fromhex(unhashed_str)
    hashedId = keccak(unhashed_byte)
    hashedId = bytes(reversed(hashedId))
    #hashedId = hashedId[0:]
    # inthash = int.from_bytes(hashedId, byteorder='big', signed='False')
    # hexString = hex(inthash)[2:]
    # print(inthash)
    # print(hexString)
    # if len(hexString) % 2 != 0:
    #     hexString = '0' + hexString
    subId = hashedId.hex()
    # subId = base58_bytes.hex()

    return '0x'+str(subId)


In [342]:
curation_mint_df['subgraphDeploymentID'] = curation_mint_df.apply(account_to_id, axis = 1)
curation_mint_df.head()

...


,id,blockNumber,blockTime,eventTime,graphAccount,subgraphNumber,nameCurator,nSignalCreated,vSignalCreated,tokensDeposited,type,subgraphDeploymentID
0,0x4c1060853752b5221b1e27c2264034d0bd5fd86c5267...,11474121,1608254015,1608254015,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,279284800875378823397,279284800875378823397,80000000000000000000000,curationMinted,0x638f6fddc855f39953f9f504bf4a97b089651e5d8529...
1,0xea7dbe2f6164530a53ac8cf9d99b46993f44d8ca8258...,11476852,1608289893,1608289893,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x0f3cf18b37c55f5a319d6cae8d8ba114cb094af0,627949250367270370,627949250367270370,360151469700892136118,curationMinted,0x638f6fddc855f39953f9f504bf4a97b089651e5d8529...
2,0xfbdb37e510284ebbeb45fbfdf28191219faf84bc31bf...,11476875,1608290226,1608290226,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x0f3cf18b37c55f5a319d6cae8d8ba114cb094af0,125503147008134479971,125503147008134479971,88216309708626528177242,curationMinted,0x638f6fddc855f39953f9f504bf4a97b089651e5d8529...
3,0xfbda17a348fb979ad36e27445d656b23458a938fadd2...,11476931,1608291049,1608291049,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0xb6916933aa0bac4b6d09260719c4055363642745,89477475325111327822,89477475325111327822,82622974825605383769993,curationMinted,0x638f6fddc855f39953f9f504bf4a97b089651e5d8529...
4,0x395da0c992967e085a68fd69825c8f629486ef10d48b...,11531681,1609017205,1609017205,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x849ed8601451dcac8012425ccd56fd1339db498c,1568268903568881,1568268903568881,1000000000000000000,curationMinted,0x638f6fddc855f39953f9f504bf4a97b089651e5d8529...


In [351]:
curation_mint_df.iloc[0]['subgraphDeploymentID']

'0x638f6fddc855f39953f9f504bf4a97b089651e5d8529364fe24d0a9ba0d6c80e'

In [349]:
unique_subgraphs[1]

'0xbbde25a2c85f55b53b7698b9476610c3d1202d88870e66502ab0076b7218f98a'

In [348]:
for i in curation_mint_df['subgraphDeploymentID']:
    if i == unique_subgraphs[1]:
        print('True')

In [ ]:
indexer_curation_mint_df = curation_mint_df[curation_mint_df['graph']]